In [3]:
from Wiki_Hurricane_Scraper import getHurricaneData
# from YouTube_Analytics import *
from FEMA_Scraper import *
import numpy as np
import pandas as pd

In [4]:
def getDisasterInfo():
    hurricaneDf = getHurricaneData()
    recentHurricanes = hurricaneDf[hurricaneDf['Season'] >= 2005]
    name = recentHurricanes['Name'].tolist()
    season = recentHurricanes['Season'].tolist()
    locations = getAllLocations()
    disastersInfo = list(zip(name, season))
    return disastersInfo, name, locations

def getFundingDf(funding, disastersInfo, name, locations):
    fundingdf = pd.DataFrame(columns=['Year'] + locations, index=name)
    for hurricane, locs in funding.items():
        for loc, funds in locs.items():
            fundingdf.at[hurricane, loc] = funds
    for name, year in disastersInfo:
        fundingdf.at[name, 'Year'] = year 
    return fundingdf

In [5]:
disastersInfo, name, locations = getDisasterInfo()

In [6]:
searchParams = {'source' : 'https://www.fema.gov/disasters',
                'location' : 'Florida',
                'incidentType' : 'Hurricane',
                'declarationType' : 'DR',
                'startMonth' : 'January',
                'startYear' : 2013,
                'endMonth' : 'December',
                'endYear' : 2018,
                'disasterName' : 'Irma'}

# funding = getAllFunding(disastersInfo, searchParams)
# np.save("fundingDict.npy", funding)

# funding = np.load('fundingDict.npy').item()
# df = getFundingDf(funding, disastersInfo, name, locations)
# df.to_csv('fundingRaised.csv')
# df = pd.read_csv('fundingRaised.csv', index_col=0)

In [12]:
# YouTubeDictDfs = getAllDisasterDfs(disastersInfo)
# for idx, (key, value) in enumerate(YouTubeDictDfs.items()):
#     value.to_csv('YouTube Dataframes/'+key+'.csv')